In [41]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import random
import glob
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from tensorflow.keras import applications
from tensorflow.keras import callbacks
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import callbacks

CALLBACK = callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=15, restore_best_weights=True)

In [42]:
import pickle

pickle_in = open("X_train_200.pickle", "rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_train_200.pickle", "rb")
Y_train = pickle.load(pickle_in)

pickle_in = open("X_test_200.pickle", "rb")
X_test = pickle.load(pickle_in)

pickle_in = open("Y_test_200.pickle", "rb")
Y_test = pickle.load(pickle_in)

In [32]:
model = tf.keras.Sequential([
    tf.keras.applications.ResNet50V2(weights=None, input_shape=X_train.shape[1:]),
    tf.keras.layers.Flatten(name="flatten"),
    tf.keras.layers.Dense(256, activation="relu", name="dense1"),
    tf.keras.layers.BatchNormalization(name="norm1"),
    tf.keras.layers.Dense(256, activation=None, name="dense4"),
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1), name="output1")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [33]:
history = model.fit(
    X_train, Y_train, batch_size=32,
    epochs=200, validation_split = 0.3, callbacks=CALLBACK)

Epoch 1/200
187/187 [==============================] - 529s 3s/step - loss: 0.7546 - val_loss: 0.8045
Epoch 2/200
187/187 [==============================] - 529s 3s/step - loss: 0.7284 - val_loss: 0.7029
Epoch 3/200
187/187 [==============================] - 530s 3s/step - loss: 0.6872 - val_loss: 0.8771
Epoch 4/200
187/187 [==============================] - 529s 3s/step - loss: 0.6271 - val_loss: 1.0087
Epoch 5/200
187/187 [==============================] - 529s 3s/step - loss: 0.5605 - val_loss: 0.6358
Epoch 6/200
187/187 [==============================] - 529s 3s/step - loss: 0.5429 - val_loss: 0.9009
Epoch 7/200
187/187 [==============================] - 529s 3s/step - loss: 0.5704 - val_loss: 0.5444
Epoch 8/200
187/187 [==============================] - 529s 3s/step - loss: 0.5400 - val_loss: 0.7541
Epoch 9/200
187/187 [==============================] - 529s 3s/step - loss: 0.5153 - val_loss: 0.6548
Epoch 10/200
187/187 [==============================] - 528s 3s/step - loss: 0.494

In [34]:
train_embs = model.predict(X_train)
test_embs = model.predict(X_test)

115/115 [==============================] - 63s 545ms/step


In [35]:
import io
import tensorflow_datasets as tfds

np.savetxt("vecs-ResNet50-d256-d128.tsv", test_embs, delimiter='\t')

out_m = io.open('meta-ResNet50-d256-d128.tsv', 'w', encoding='utf-8')
[out_m.write(str(x) + "\n") for x in Y_test]
out_m.close()


try:
  from google.colab import files
  files.download('vecs-ResNet50-d256-d256.tsv')
  files.download('meta-ResNet50-d256-d256.tsv')
except:
  pass

In [36]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(train_embs, Y_train)
y_test_pred_knn = knn.predict_proba(test_embs)


In [37]:
predict = []
for pred in y_test_pred_knn:
    if(pred[0]>=pred[1]):
        predict.append(0)
    else:
        predict.append(1)

In [38]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range (len(predict)):
    if predict[i] == 0 and Y_test[i] == 0:
        TN += 1
    elif predict[i] == 1 and Y_test[i] == 1:
        TP += 1
    elif predict[i] == 0 and Y_test[i] == 1:
        FN += 1
    elif predict[i] == 1 and Y_test[i] == 0:
        FP += 1

In [39]:
czulosc = TP / (TP + FN)
swoistosc = TN / (TN + FP)

print(czulosc)
print(swoistosc)

0.8728165938864629
0.9470234844347352


In [40]:
model.save('ResNet50-d256-d256')

INFO:tensorflow:Assets written to: ResNet50-d256-d256\assets


INFO:tensorflow:Assets written to: ResNet50-d256-d256\assets
